In [14]:
import joblib
import pyrealsense2 as rs
import numpy as np
import cv2
from skimage.measure import regionprops, perimeter
from skimage import io, morphology
clf = joblib.load('meu_modelo.joblib')

In [16]:
game_rules = {"pedra": cv2.resize(cv2.imread("response_images/paper.png",cv2.IMREAD_COLOR), (640,480)),
              "papel": cv2.resize(cv2.imread("response_images/scissors.png",cv2.IMREAD_COLOR), (640,480)),
              "tesoura": cv2.resize(cv2.imread("response_images/rock.png",cv2.IMREAD_COLOR), (640,480)),
              "void": cv2.resize(cv2.imread("response_images/void.png",cv2.IMREAD_COLOR), (640,480))}

In [17]:
cv2.imshow("test", game_rules["void"])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
def get_feats(img):
    props = regionprops(img, coordinates='xy')[0]
    eccentricity = props['eccentricity']
    hu_moments = props['moments_hu']
    area = props['area']
    perimeter = props['perimeter']
    if perimeter == 0.0:
        perimeter = 1.0
    compactness = 4.0*np.pi*area/perimeter
    feats = [eccentricity, compactness] + list(hu_moments)
    return [feats]

In [19]:
slem =  morphology.disk(4)
font = cv2.FONT_HERSHEY_SIMPLEX
colorT = 255

In [21]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
profile = pipeline.start(config)

depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

clipping_distance_in_meters = .4
clipping_distance = clipping_distance_in_meters / depth_scale

# Streaming loop
try:
    while True:
        
        frames = pipeline.wait_for_frames()
        
        depth_frame = frames.get_depth_frame()

        if not depth_frame:
            continue

        depth_image = np.asanyarray(depth_frame.get_data())
        depth_corrected = np.where((depth_image > clipping_distance)| (depth_image <= 0), 0, 255)
        depth_corrected = np.uint8(depth_corrected)
        after = morphology.closing(depth_corrected,selem=slem)
        text = "void"
        if(np.count_nonzero(after) > 500):
            feats = get_feats(after)
            text  = clf.predict(feats)[0]
        
        cv2.putText(after, text,(50,50), font, 1, colorT, 2, cv2.LINE_AA)
        
        cv2.imshow("Output", after)
        cv2.imshow("Response", game_rules[text])
        key = cv2.waitKey(1)
        
        
        
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
finally:
    pipeline.stop()
    cv2.destroyAllWindows()

In [7]:
print(prediction)

NameError: name 'prediction' is not defined